# Text generation with DeepSeek-R1-Distill-Llama-8B

In [2]:
import requests
import concurrent.futures
import subprocess
import platform
import shutil


model = "deepseek-r1:8b"

def submit_prompt(model, prompt):
    url = "http://localhost:11434/api/generate"
    standard_response = "I am sorry, I cannot answer that question. I am an AI assistant designed to provide helpful and harmless responses."
    i = 0
    output = standard_response
    
    while standard_response in output and i < 5:
        i += 1
        data = {
            "model": model,
            "prompt": prompt,
            "stream": False
        }

        response = requests.post(url, json=data)
        think = ""
        output = response.json()['response']
        last_occurrence = output.rfind('</think>')
        if last_occurrence != -1:
            think = output[:last_occurrence + len('</think>')]
            output = output[last_occurrence + len('</think>'):].lstrip()

    return think, output

def process_prompt_batch(model, prompts, output_file, max_workers=5):
    with open(output_file, "w", encoding="utf-8") as f:
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_prompt = {executor.submit(submit_prompt, model, prompt): prompt for prompt in prompts}
            for future in concurrent.futures.as_completed(future_to_prompt):
                prompt = future_to_prompt[future]
                try:
                    think, output = future.result()
                    result = {"prompt": prompt, "response": output, "thought_process": think}
                    f.write(json.dumps(result, ensure_ascii=False) + "\n")
                except Exception as e:
                    print(f"Errore durante l'elaborazione del prompt '{prompt}': {e}")

def process_prompts(model, prompts_df):
    with open('./partial_results.json', 'w') as f:
        if 'response' not in prompts_df.columns:
            prompts_df['response'] = None
        for prompt in prompts_df['prompt'].to_list():
            think, output = submit_prompt(model, prompt)
            prompts_df.loc[prompts_df['prompt'] == prompt, 'thought_process'] = think
            prompts_df.loc[prompts_df['prompt'] == prompt, 'response'] = output
            prompts_df.loc[prompts_df['prompt'] == prompt].to_json(f, orient='records', lines=True)
    
    return prompts_df

def is_ollama_installed():
    return shutil.which("ollama") is not None or shutil.which("ollama.exe") is not None

def is_ollama_running(host='localhost', port=11434):
    """Verifica se il server di Ollama è in esecuzione."""
    try:
        response = requests.get(f'http://{host}:{port}/api/status', timeout=2)
        return response.status_code == 200
    except requests.RequestException:
        return False

def start_ollama():
    if not is_ollama_installed():
        raise RuntimeError("Ollama non è installato sul sistema. Impossibile avviare il servizio.")
        
    if not is_ollama_running():
        print("Ollama non è in esecuzione. Avvio...")
        system = platform.system()
        if system == "Windows":
            subprocess.Popen(['ollama.exe', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, shell=True)
        elif system == "Linux" or system == "Darwin":  # macOS è "Darwin"
            subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        else:
            print("Sistema operativo non supportato.")
    else:
        print("Ollama è già in esecuzione.")

start_ollama()


Ollama non è in esecuzione. Avvio...


In [ ]:
import pandas as pd

# Load prompts from a JSON file
prompts_df = pd.read_json('./prompts.json')
results = process_prompt_batch(model, prompts_df, 'responses.json')